In [1]:
## FUNCTIONS FOR USING .DAT FILES FROM CW3E MET SITES FOR ANALYSIS
# By Hillary Beckmeyer (& nan_check fxn augmented from fxn by Ava Cooper)
# 25.AUG.2022
#
# SOME ALTERATIONS MAY BE NEEDED DEPENDENT ON WHAT ASPECTS OF THE DATA ARE BEING USED. THIS IS FOR TEMP (˚C), 
# RH (%), RAW BP (mb), MSLP (mb), WIND V1 (M/S), WIND MAX (M/S), 'SLR (W/M^2), PRECIP (mm), AND MIN BATTERY (V).
#
# ADDED ARE SOME FUNCTIONS FOR SOIL TEMP, PRESSURE, AND MOISTURE (AT DEPTHS OF 5CM, 10CM, 15CM, 20CM, 50CM, AND 100CM).
#
# CAN BE USED FOR GRAPHING VARIABLES AND COMPARATIVE ANALYSIS. ALSO FOR FINDING THE VARIABLES NEEDED FOR THE CLEARSKY
# CALCULATOR.


In [2]:
# IMPORTING PACKAGES THAT MAY BE NEEDED
%matplotlib inline
import matplotlib as plt
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import matplotlib.dates as mdates
import matplotlib.colors as colors
from matplotlib import cm, colors
import pandas as pd
import seaborn as sns
import pdb, sys, glob, os, shutil, netCDF4, argparse, csv

np.set_printoptions(suppress = True, formatter = {'float_kind':'{:0.2f}'.format})

In [3]:
# set root directory to surface met archive: THIS WILL BE WHEREVER SURF MET DATA LIVES ON YOUR COMPUTER
rd = "~/Downloads/CW3E_SurfaceMet_Archive/"

# THIS IS FOR SETTING THE DIRECTORY FOR ALL FILES (IF THEY'RE IN THE SAME FOLDER)
%cd $rd

/Users/hillarybeckmeyer/Downloads/CW3E_SurfaceMet_Archive


In [4]:
# FUNCTIONS FOR IMPORTING THE FILES AS A PANDAS DATAFRAME

def imp(file):
    f = pd.read_csv(file, sep = ',', header = None, index_col = 0, skiprows = 4, \
                    na_values = "NAN", keep_default_na = False, parse_dates = [0])
    print(f.shape)
    return f

# THERE WAS AN ERROR WITH THE HRD FILE BUT A QUICK FIX WAS TO SKIP THE FIRST 350,000 ROWS. ONLY USE IF YOU NEED
# MORE RECENT DATA AND DON'T HAVE THE TIME TO FIND THE ERRORS
def hrd(file):
    f = pd.read_csv(file, sep = ',', header = None, index_col = 0, skiprows = 350000, \
                    na_values = "NAN", keep_default_na = False, parse_dates = [0])
    print(f.shape)
    return f

# CHECKING THE TITLES W/ NO LABELS CUT OUT - NOT FOR GENERAL USE
def imp2(file):
    f = pd.read_csv(file, sep = ',', header = None, index_col = 0, skiprows = 1, \
                    na_values = "NAN", keep_default_na = False, parse_dates = [0])
    print(f.shape)
    return f

In [5]:
## FOR 2 MIN DATA:
## MAKING A NaN CHECK FXN TO FILL MISSING TIMESTAMPS & REDUCE # OF VARIABLES IN EACH DATAFRAME

# DEFINE FXN nan_check - TAKES 2 MIN DATAFRAME INPUT: SITE_FILE
def nan_check(SITE_FILE):
    
    # picking columns of interest
    use = SITE_FILE[[2,5,8,9,12,15,17,23,25]]
    # resample to 2 min (data are already 2 min so this fills in missing timestamps)
    met = use.resample('2T').mean()
    # rename columns
    met.columns = ['Temp (\N{degree sign}C)', 'RH (%)', 'Raw BP (mb)', 'BP MSL (mb)', 'Wnd V1', \
                       'Wnd max', 'SLR (W/m\N{superscript two})', 'PPT (mm)', 'Min Batt (V)']
    
    # count the # of NaNs per column
    nan_count = met.isnull().sum()
    
    # find the actual timestamps where any value in the dataframe is NaN
    ind = np.where(np.isnan(met))[0]
    
    # grab those timestamps from the data, drop repeated timestamps 
    # (occurs when multiple variables are NaN @ the same time)
    bad_idx = met.iloc[ind].index.drop_duplicates(keep = 'first')
    
    # fxn outputs 3 things: dataframe for entire time, array for bad index, & total count of NaNs per var
    return(met, bad_idx, nan_count)

In [6]:
## FOR SOIL DATA:
## MAKING A NaN CHECK FXN TO FILL MISSING TIMESTAMPS & REDUCE # OF VARIABLES IN EACH DATAFRAME

# DEFINE FXN nan_check - TAKES 2 MIN DATAFRAME INPUT: SITE_FILE
def nan_check_soil(SITE_FILE):
    
    # grab columns of interest
    use = SITE_FILE[[2,3,4,5,6,7,20,21,22,23,24,25,26,27,28,29,30,31]]
    # resample to 2 minutes (data are already 2 min so this fills in missing timestamps)
    met = use.resample('2T').mean()
    # rename columns
    met.columns = ['SoilTmp_5_C_Avg',  'SoilTmp_10_C_Avg', 'SoilTmp_15_C_Avg', 'SoilTmp_20_C_Avg', \
                       'SoilTmp_50_C_Avg','SoilTmp_100_C_Avg', 'PA_uS_5', 'PA_uS_10', 'PA_uS_15', \
                       'PA_uS_20', 'PA_uS_50', 'PA_uS_100', 'VWC_5', 'VWC_10', 'VWC_15', 'VWC_20', \
                       'VWC_50', 'VWC_100']
    
    # count the # of NaNs per column
    nan_count = met.isnull().sum()
    
    # find the actual timestamps where any value in the dataframe is NaN
    ind = np.where(np.isnan(met))[0]
    
    # grab those timestamps from the data, drop repeated timestamps 
    # (occurs when multiple variables are NaN @ the same time)
    bad_idx = met.iloc[ind].index.drop_duplicates(keep = 'first')
    
    # fxn outputs 3 things: dataframe for entire time, array for bad index, & total count of NaNs per var
    # will most likely only need the first output
    return(met, bad_idx, nan_count)

In [7]:
# SURFACE MET FILE UPLOAD: Use fxn to import file. 
# CUSTOMIZE SITE AND FILE_NAME BASED ON WHICH SITE(S) YOU ARE LOOKING AT.
#SITE = imp('FILE_NAME')

GPO = imp('GranitePortal_TwoMin.dat'            )
POR = imp('Portola_TwoMin.dat'                  )
BVS = imp('BrownsValleySchool_TwoMin.dat'       )
DLA = imp('CR1000X_Downieville_RF407_TwoMin.dat')
SKY = imp('Skyline_TwoMinWS.dat'                )
WPO = imp('CR1000X_WPO_TwoMin.dat'              )
WDG = imp('WindyGap_TwoMinWS.dat'               )
CAT = imp('CAT_CR1000X_TwoMin.dat'              )
NCM = imp('NorthCowMountain_TwoMinWS.dat'       )
LBH = imp('LowerBathHouse_TwoMin.dat'           )
FRC = imp('FeatherRiverCollege_TwoMin.dat'      )
BCC = imp('BoyesCreekCanyon_TwoMinWS 2.dat'     )
NBB = imp('CR1000X_NBB_radio_TwoMin.dat'        )
HDC = imp('HellsDelight_TwoMinWS.dat'           )
PVN = imp('PotterValleyNorth_TwoMinWS.dat'      )
HRD = hrd('CR1000X_HRD_Radio_TwoMin.dat'        )
DRW = hrd('Deerwood_TwoMinWS.dat'               )
SOD = hrd('CR1000X_SOD_TwoMin.dat'              )

(43228, 27)
(154032, 27)
(305223, 27)
(706965, 27)
(696106, 27)
(247508, 27)
(1033996, 27)
(616755, 27)
(1032819, 27)
(200147, 27)
(732692, 27)
(968784, 27)
(708184, 27)
(922978, 27)
(987355, 27)
(72523, 27)
(683644, 27)
(234504, 27)


In [8]:
# SOIL FILE UPLOAD: Use fxn to import file.

# CUSTOMIZE SITE AND FILE_NAME BASED ON WHICH SITE(S) YOU ARE LOOKING AT.
# I like to differentiate by adding an "S" to the end of the dataframe name if it is a soil file vs. a surf. met. one.
# That way, if I'm using both, it's easy to differentiate.

#SITE_S = imp('FILE_NAME')

In [9]:
# SURFACE MET DATA: Edit depending on the sites you are looking at, or skip if only viewing one site
site_list = [BCC, BVS, CAT, DRW, DLA, FRC, GPO, HDC, HRD, LBH, NBB, NCM, POR, PVN, SKY, SOD, WDG, WPO]
site_name = ['BCC', 'BVS', 'CAT', 'DRW', 'DLA', 'FRC', 'GPO', 'HDC', 'HRD', 'LBH', 'NBB', 'NCM', 'POR', \
              'PVN', 'SKY', 'SOD', 'WDG', 'WPO']

mets, badidxs, nans = [], [], []
for x in range(len(site_list)):
    met, badidx, nan = nan_check(site_list[x])
    mets.append(met), badidxs.append(badidx), nans.append(nan)

for x in range(len(mets)):
    mets[x].index.rename(site_name[x], inplace = True)

In [10]:
for i in range(len(mets)):
    print(mets[i].index.name)

BCC
BVS
CAT
DRW
DLA
FRC
GPO
HDC
HRD
LBH
NBB
NCM
POR
PVN
SKY
SOD
WDG
WPO


In [11]:
# SURFACE MET DATA: If just looking at a singular site, and changing index name to 'Time'
#SITE_met, SITE_badidx, SITE_nan = nan_check(SITE)
#SITE_met.index.rename('Time', inplace = True)

WDG_met, WDG_badidx, WDG_nan = nan_check(WDG)
WDG_met.index.rename('Time', inplace = True)

NCM_met, NCM_badidx, NCM_nan = nan_check(NCM)
NCM_met.index.rename('Time', inplace = True)

HDC_met, HDC_badidx, HDC_nan = nan_check(HDC)
HDC_met.index.rename('Time', inplace = True)

DLA_met, DLA_badidx, DLA_nan = nan_check(DLA)
DLA_met.index.rename('Time', inplace = True)

SKY_met, SKY_badidx, SKY_nan = nan_check(SKY)
SKY_met.index.rename('Time', inplace = True)

FRC_met, FRC_badidx, FRC_nan = nan_check(FRC)
FRC_met.index.rename('Time', inplace = True)

DRW_met, DRW_badidx, DRW_nan = nan_check(DRW)
DRW_met.index.rename('Time', inplace = True)

In [12]:
# SOIL DATA: Apply nan_check_soil fxn to site and change index name to 'Time'
#SITE_met, SITE_badidx, SITE_nan = nan_check_soil(SITE)
#SITE_met.index.rename('Time', inplace = True)

In [13]:
## RENAMING THE VARIABLES TO SIMPLIFY
# surface met vars
tp, rh, bp, msl, v1, wmax, sr, ppt, batt = 'Temp (\N{degree sign}C)', 'RH (%)', 'Raw BP (mb)', 'BP MSL (mb)', \
    'Wnd V1', 'Wnd max', 'SLR (W/m\N{superscript two})', 'PPT (mm)', 'Min Batt (V)'

# Soil vars
t5, t10, t15, t20, t50, t100, p5, p10, p15, p20, p50, p100, m5, m10, m15, m20, m50, m100 = 'SoilTmp_5_C_Avg',  \
    'SoilTmp_10_C_Avg', 'SoilTmp_15_C_Avg', 'SoilTmp_20_C_Avg','SoilTmp_50_C_Avg','SoilTmp_100_C_Avg', 'PA_uS_5',\
    'PA_uS_10', 'PA_uS_15','PA_uS_20', 'PA_uS_50', 'PA_uS_100','VWC_5', 'VWC_10', 'VWC_15', 'VWC_20', \
    'VWC_50', 'VWC_100'

## PUTTING VARIABLES INTO LIST
#impvals = most important vals for ClearSky model
impvals = [tp, rh, sr]
vals    = [tp, rh, bp, msl, v1, wmax, sr, ppt, batt]

# Separating soil vars by type
soil_temp     = [t5, t10, t15, t20, t50, t100] 
soil_p        = [p5, p10, p15, p20, p50, p100]
soil_moisture = [m5, m10, m15, m20, m50, m100]

In [14]:
# Fxn to plot any list of soil variables

def allSoil(site, types, word, loc):
    plt.rcParams["figure.figsize"] = [20.00, 8.0]
    plt.rcParams["figure.autolayout"] = True
    fig, ax = plt.subplots(1)
    fig.autofmt_xdate()
    # run through list of depths, if wanting to plot multiple vars
    for i in range(len(types)): 
        plt.plot(site[types[i]], label = types[i])
    plt.xlabel('Date', fontweight = 'bold', fontsize = 16)
    plt.ylabel(word, fontweight = 'bold', fontsize = 16)
    # can change ymin and ymax depending on season or scope 
    ax.set_ylim(ymin = 0, ymax = 45)
    xfmt = mdates.DateFormatter('%D')
    ax.xaxis.set_major_formatter(xfmt)
    plt.title(word+loc, fontweight = 'bold', fontsize = '20')
    plt.legend(loc = 'upper left')
    plt.show()

# Fxn specifying soil temp
def allTemp(site, loc):
    allSoil(site, soil_temp, 'Soil Temp (\N{degree sign}C)', loc) 
    # allSoil(site, soil_temp[0:1], 'Soil Temp (\N{degree sign}C)', loc) 
    
# Fxn specifying soil moisture
def allMoisture(site, loc):
    allSoil(site, soil_moisture, 'Soil Moisture', loc) 
    # allSoil(site, soil_moisture[0:1], 'Soil Moisture', loc) 

In [15]:
b = NCMS_met["2018-03-30 00:00":"2022-08-24 07:00"]
allTemp(b, ' @ NCM')

NameError: name 'NCMS_met' is not defined

In [ ]:
ticky = np.arange(0, 1300, step = 100)

# Fxn to plot variables over a span of time
def VARday(var, kind, words):
    sns.set_theme(style = "ticks", font_scale = 1.25)
    sns.despine(offset = 10, trim = True)
    ax = plt.gca()
    var[kind].plot(color = 'm', figsize = (15,4))
    plt.title(str(kind)+': '+words)
    plt.xlabel('Date')
    plt.ylabel(kind)
    ax.set_ylim(ymin = 0, ymax = 1200)
    plt.yticks(ticky)
    plt.show()

# Fxn specifying solar rad to plot
def SLR(var, words):
    VARday(var, sr, words)
    
def graphALL(site, v):
    for x in range(len(site)):
        VARday((site[x])[v], v, str())
    
def allSLR(var):
    for i in range(len(var)):
        SLR((var[i]), str(var[i].index.date[0]))
    
#finding the maximum value of a variable for a specified timeframe
def maxVAR(var):    
    var_day = var
    h = var_day.shape[0]
    
    high = 0
    time_x = 0
    
    for x in range(h):
        if(var[x] > high):
            high = var[x]
            time_x = var.index[x]
            hour_x = var.index.time[x]
            date_x = var.index.date[x]
        else:
            continue
            
    return high, time_x, hour_x, date_x

# Fxn printing the maximum date, time, and value of a variable
def prntMAX(var, num, end):
    high, time_x, hour_x, date_x = maxVAR(var)
    print('Peak', str((GPO_met.columns)[num]), 'is on', date_x, 'at', hour_x, 'UTC:', high, end)
    return high, time_x, hour_x, date_x

# Fxn specifying solar rad as the variable to print
def solar(var):
    maxi, ind, hour, date = prntMAX(var, 6, 'W/m\N{superscript two}')
    return maxi, ind, hour, date
    
# Fxn finding the other variable quantities at max solar rad time
def allVAR(var, loc):
    h = len(loc.columns)
    high, time_x, hour_x, date_x = maxVAR(var)
    print(time_x,'\n')
    vals = np.array([], dtype = np.float64)
    x = 0
    for col in loc.columns: 
        if x < h:
            b = loc.columns[x]
            vals = np.append(vals, loc[b][time_x])
            print(b, ':', loc[b][time_x])
            x += 1
        else:
            break;
            
    print('\n')
    return vals

def clearsky(site, kind, vals):
    A, B, C, D, E, F, G, H, I, J, K = np.array([]), np.array([]), np.array([]), np.array([]), \
    np.array([]), np.array([]), np.array([]), np.array([]), np.array([]), np.array([]), np.array([])
    Z = ()
    for q in range(len(site)):
        a = (site[q])[kind].max()
        b = (site[q])[kind].idxmax()
        A = np.append(A, a)
        B = np.append(B, b)
        print('\n')
        print(b)
        for r in range(len(vals)):
            print(vals[r], '=', (site[q])[vals[r]][b])
            if vals[r] == 'Temp (\N{degree sign}C)':
                c = (site[q])[vals[r]][b]
                C = np.append(C, c)
            elif vals[r] == 'RH (%)':
                d = (site[q])[vals[r]][b]
                D = np.append(D, d) 
            elif vals[r] == 'Raw BP (mb)':
                e = (site[q])[vals[r]][b]
                E = np.append(E, e)
            elif vals[r] == 'BP MSL (mb)':
                f = (site[q])[vals[r]][b]
                F = np.append(F, f)
            elif vals[r] == 'Wnd V1':
                g = (site[q])[vals[r]][b]
                G = np.append(G, g)
            elif vals[r] == 'Wnd max':
                h = (site[q])[vals[r]][b]
                H = np.append(H, h)
            elif vals[r] == 'SLR (W/m\N{superscript two})':
                i = (site[q])[vals[r]][b]
                I = np.append(I, i)
            elif vals[r] == 'PPT (mm)':
                j = (site[q])[vals[r]][b]
                J = np.append(J, j)
            elif vals[r] == 'Min Batt (V)':
                k = (site[q])[vals[r]][b]
                K = np.append(K, k)
            else:
                break
        Z = (A, B, C, D, E, F, G, H, I, J, K)
    return(Z)

def csky(site):
    x = clearsky(site, sr, impvals)
    return(x)

In [ ]:
#Set fxn timespans to ranges you'd like to look at
def groups(site):
    A = site["2022-04-01 07:00":"2022-04-10 07:00"][sr]
    B = site["2022-04-10 7:00":"2022-04-20 07:00"][sr]
    C = site["2022-04-20 7:00":"2022-04-30 07:00"][sr]
    D = site["2022-04-30 7:00":"2022-05-09 07:00"][sr]
    G = [A, B, C, D]
    return(G)

def foursome(group):
    for i in range(len(group)):
        SLR(group[i], ' from '+str(group[i].index.date[0])+' to '+str(group[i].index.date[-1]))

In [ ]:
## Pick specific dates and times you would like to look at for each site
print(mets[6].index.name)

gpo_whole = mets[6]
gpo_set = mets[6]["2022-04-01 07:00":"2022-05-09 18:02"]

gpo0    = mets[6]['2022-04-01 13:00':'2022-04-02 03:00']
gpo1    = mets[6]['2022-04-05 13:00':'2022-04-06 03:00']
gpo2    = mets[6]['2022-04-06 13:00':'2022-04-07 03:00']
gpo3    = mets[6]['2022-04-09 13:00':'2022-04-10 03:00']
gpo4    = mets[6]['2022-04-17 13:00':'2022-04-18 03:00']
gpo5    = mets[6]['2022-04-25 13:00':'2022-04-26 03:00']
gpo6    = mets[6]['2022-04-28 13:00':'2022-04-29 03:00']
gpo7    = mets[6]['2022-04-30 13:00':'2022-05-01 03:00']
gpo8    = mets[6]['2022-05-01 13:00':'2022-05-02 03:00']
gpo9    = mets[6]['2022-05-03 13:00':'2022-05-04 03:00']
gpo10   = mets[6]['2022-05-04 13:00':'2022-05-05 03:00']

# Making a list combining all the days and the name of the site will come in handy down the line.
gpox = [gpo0, gpo1, gpo2, gpo3, gpo4, gpo5, gpo6, gpo7, gpo8, gpo9, gpo10]
gpoz = [mets[6].index.name, "Granite Portal", gpox]

In [ ]:
## Pick specific dates and times you would like to look at for each site

print(mets[12].index.name)

por_whole = mets[12]
por_set = mets[12]['2022-04-01 07:00':'2022-05-09 18:02']

por0    = mets[12]['2022-04-01 13:00':'2022-04-02 03:00']
por1    = mets[12]['2022-04-05 13:00':'2022-04-06 03:00']
por2    = mets[12]['2022-04-06 13:00':'2022-04-07 03:00']
por3    = mets[12]['2022-04-09 13:00':'2022-04-10 03:00']
por4    = mets[12]['2022-04-17 13:00':'2022-04-18 03:00']
por5    = mets[12]['2022-04-18 13:00':'2022-04-19 03:00']
por6    = mets[12]['2022-04-24 13:00':'2022-04-25 03:00']
por7    = mets[12]['2022-04-27 13:00':'2022-04-28 03:00']
por8    = mets[12]['2022-05-01 13:00':'2022-05-02 03:00']
por9    = mets[12]['2022-05-03 13:00':'2022-05-04 03:00']
por10   = mets[12]['2022-05-04 13:00':'2022-05-05 03:00']

porx = [por0, por1, por2, por3, por4, por5, por6, por7, por8, por9, por10]
porz = [mets[12].index.name, "Portola", porx]

In [ ]:
## Pick specific dates and times you would like to look at for each site

print(mets[1].index.name)

bvs_whole = mets[1]
bvs_set   = mets[1]["2022-04-01 07:00":"2022-05-09 18:02"]

bvs0      = mets[1]['2022-04-01 13:00':'2022-04-02 03:00']
bvs1      = mets[1]['2022-04-05 13:00':'2022-04-06 03:00']
bvs2      = mets[1]['2022-04-06 13:00':'2022-04-07 03:00']
bvs3      = mets[1]['2022-04-09 13:00':'2022-04-10 03:00']
bvs4      = mets[1]['2022-04-17 13:00':'2022-04-18 03:00']
bvs5      = mets[1]['2022-04-23 13:00':'2022-04-24 03:00']
bvs6      = mets[1]['2022-04-26 13:00':'2022-04-27 03:00']
bvs7      = mets[1]['2022-04-28 13:00':'2022-04-29 03:00']
bvs8      = mets[1]['2022-05-01 13:00':'2022-05-02 03:00']
bvs9      = mets[1]['2022-05-03 13:00':'2022-05-04 03:00']
bvs10     = mets[1]['2022-05-04 13:00':'2022-05-05 03:00']

bvsx = [bvs0, bvs1, bvs2, bvs3, bvs4, bvs5, bvs6, bvs7, bvs8, bvs9, bvs10]
bvsz = [mets[1].index.name, "Browns Valley School", bvsx]

In [ ]:
## Pick specific dates and times you would like to look at for each site

print(mets[4].index.name)

dla_whole = mets[4]
dla_set = mets[4]["2022-04-01 07:00":"2022-05-09 18:02"]

dla0    = mets[4]['2022-04-01 13:00':'2022-04-02 03:00']
dla1    = mets[4]['2022-04-05 13:00':'2022-04-06 03:00']
dla2    = mets[4]['2022-04-06 13:00':'2022-04-07 03:00']
dla3    = mets[4]['2022-04-09 13:00':'2022-04-10 03:00']
dla4    = mets[4]['2022-04-17 13:00':'2022-04-18 03:00']
dla5    = mets[4]['2022-04-24 13:00':'2022-04-25 03:00']
dla6    = mets[4]['2022-05-01 13:00':'2022-05-02 03:00']
dla7    = mets[4]['2022-05-03 13:00':'2022-05-04 03:00']
dla8    = mets[4]['2022-05-04 13:00':'2022-05-05 03:00']

dlax = [dla0, dla1, dla2, dla3, dla4, dla5, dla6, dla7, dla8]
dlaz = [mets[4].index.name, "Downieville", dlax]

In [ ]:
## Pick specific dates and times you would like to look at for each site

print(mets[14].index.name)

sky_whole = mets[14]
sky_set   = mets[14]["2022-04-01 07:00":"2022-05-09 18:02"]

sky0      = mets[14]['2022-04-01 13:00':'2022-04-02 03:00']
sky1      = mets[14]['2022-04-02 13:00':'2022-04-03 03:00']
sky2      = mets[14]['2022-04-05 13:00':'2022-04-06 03:00']
sky3      = mets[14]['2022-04-06 13:00':'2022-04-07 03:00']
sky4      = mets[14]['2022-04-17 13:00':'2022-04-18 03:00']
sky5      = mets[14]['2022-04-23 13:00':'2022-04-24 03:00']
sky6      = mets[14]['2022-04-26 13:00':'2022-04-27 03:00']
sky7      = mets[14]['2022-04-27 13:00':'2022-04-28 03:00']
sky8      = mets[14]['2022-05-01 13:00':'2022-05-02 03:00']
sky9      = mets[14]['2022-05-03 13:00':'2022-05-04 03:00']
sky10     = mets[14]['2022-05-04 13:00':'2022-05-05 03:00']

skyx = [sky0, sky1, sky2, sky3, sky4, sky5, sky6, sky7, sky8, sky9, sky10]
skyz = [mets[14].index.name, "Skyline Harvest", skyx]

In [ ]:
## Pick specific dates and times you would like to look at for each site

print(mets[-1].index.name)

wpo_whole = mets[-1]
wpo_set   = mets[-1]["2022-04-01 07:00":"2022-05-09 18:02"]

wpo0      = mets[-1]['2022-04-01 13:00':'2022-04-02 03:00']
wpo1      = mets[-1]['2022-04-05 13:00':'2022-04-06 03:00']
wpo2      = mets[-1]['2022-04-06 13:00':'2022-04-07 03:00']
wpo3      = mets[-1]['2022-04-09 13:00':'2022-04-10 03:00']
wpo4      = mets[-1]['2022-04-12 13:00':'2022-04-13 03:00']
wpo5      = mets[-1]['2022-04-17 13:00':'2022-04-18 03:00']
wpo6      = mets[-1]['2022-04-23 13:00':'2022-04-24 03:00']
wpo7      = mets[-1]['2022-04-28 13:00':'2022-04-29 03:00']
wpo8      = mets[-1]['2022-05-01 13:00':'2022-05-02 03:00']
wpo9      = mets[-1]['2022-05-03 13:00':'2022-05-04 03:00']
wpo10     = mets[-1]['2022-05-04 13:00':'2022-05-05 03:00']

wpox = [wpo0, wpo1, wpo2, wpo3, wpo4, wpo5, wpo6, wpo7, wpo8, wpo9, wpo10]
wpoz = [mets[-1].index.name, "West Portal", wpox]

In [ ]:
## Pick specific dates and times you would like to look at for each site

print(mets[-2].index.name)

wdg_whole = mets[-2]
wdg_set   = mets[-2]["2022-04-01 07:00":"2022-05-09 18:02"]

wdg0      = mets[-2]['2022-04-05 13:00':'2022-04-06 03:00']
wdg1      = mets[-2]['2022-04-06 13:00':'2022-04-07 03:00']
wdg2      = mets[-2]['2022-04-09 13:00':'2022-04-10 03:00']
wdg3      = mets[-2]['2022-04-17 13:00':'2022-04-18 03:00']
wdg4      = mets[-2]['2022-04-23 13:00':'2022-04-24 03:00']
wdg5      = mets[-2]['2022-04-24 13:00':'2022-04-25 03:00']
wdg6      = mets[-2]['2022-04-26 13:00':'2022-04-27 03:00']
wdg7      = mets[-2]['2022-04-27 13:00':'2022-04-28 03:00']
wdg8      = mets[-2]['2022-05-01 13:00':'2022-05-02 03:00']
wdg9      = mets[-2]['2022-05-03 13:00':'2022-05-04 03:00']
wdg10     = mets[-2]['2022-05-04 13:00':'2022-05-05 03:00']

wdgx = [wdg0, wdg1, wdg2, wdg3, wdg4, wdg5, wdg6, wdg7, wdg8, wdg9, wdg10]
wdgz = [mets[-2].index.name, "Windy Gap", wdgx]

In [ ]:
## Pick specific dates and times you would like to look at for each site

print(mets[8].index.name)

hrd_whole = mets[8]
hrd_set   = mets[8]["2022-04-01 07:00":"2022-05-09 18:02"]

hrd0      = mets[8]['2022-04-05 13:00':'2022-04-06 03:00']
hrd1      = mets[8]['2022-04-06 13:00':'2022-04-07 03:00']
hrd2      = mets[8]['2022-04-08 13:00':'2022-04-09 03:00']
hrd3      = mets[8]['2022-04-09 13:00':'2022-04-10 03:00']
hrd4      = mets[8]['2022-04-12 13:00':'2022-04-12 21:00']
hrd5      = mets[8]['2022-04-17 13:00':'2022-04-18 03:00']
hrd6      = mets[8]['2022-04-23 13:00':'2022-04-24 03:00']
hrd7      = mets[8]['2022-04-28 13:00':'2022-04-29 03:00']
hrd8      = mets[8]['2022-05-01 13:00':'2022-05-01 20:40']
hrd9      = mets[8]['2022-05-03 13:00':'2022-05-04 03:00']
hrd10     = mets[8]['2022-05-04 13:00':'2022-05-05 03:00']

hrdx = [hrd0, hrd1, hrd2, hrd3, hrd4, hrd5, hrd6, hrd7, hrd8, hrd9, hrd10]
hrdz = [mets[8].index.name, "Henness Ridge Drive", hrdx]

In [ ]:
## Pick specific dates and times you would like to look at for each site

print(mets[2].index.name)

cat_whole = mets[2]
cat_set   = mets[2]["2022-04-01 07:00":"2022-05-09 18:02"]

cat0      = mets[2]['2022-04-06 13:00':'2022-04-07 03:00']
cat1      = mets[2]['2022-04-07 13:00':'2022-04-08 03:00']
cat2      = mets[2]['2022-04-08 13:00':'2022-04-09 03:00']
cat3      = mets[2]['2022-04-12 13:00':'2022-04-13 03:00']
cat4      = mets[2]['2022-04-13 13:00':'2022-04-14 03:00']
cat5      = mets[2]['2022-04-23 13:00':'2022-04-24 03:00']
cat6      = mets[2]['2022-04-24 13:00':'2022-04-25 03:00']
cat7      = mets[2]['2022-04-25 13:00':'2022-04-26 03:00']
cat8      = mets[2]['2022-04-30 13:00':'2022-05-01 03:00']
cat9      = mets[2]['2022-05-04 13:00':'2022-05-05 03:00']
cat10     = mets[2]['2022-05-06 13:00':'2022-05-07 03:00']

catx = [cat0, cat1, cat2, cat3, cat4, cat5, cat6, cat7, cat8, cat9, cat10]
catz = [mets[2].index.name, "Catalina", catx]

In [ ]:
## Pick specific dates and times you would like to look at for each site

print(mets[3].index.name)

drw_whole = mets[3]

drw0      = mets[3]['2022-04-01 13:00':'2022-04-02 03:00']
drw1      = mets[3]['2022-04-05 13:00':'2022-04-06 03:00']
drw2      = mets[3]['2022-04-06 13:00':'2022-04-07 03:00']
drw3      = mets[3]['2022-04-09 13:00':'2022-04-10 03:00']
drw4      = mets[3]['2022-04-17 13:00':'2022-04-18 03:00']
drw5      = mets[3]['2022-04-24 13:00':'2022-04-25 03:00']
drw6      = mets[3]['2022-04-26 13:00':'2022-04-27 03:00']
drw7      = mets[3]['2022-04-27 13:00':'2022-04-28 03:00']
drw8      = mets[3]['2022-05-01 13:00':'2022-05-02 03:00']
drw9      = mets[3]['2022-05-03 13:00':'2022-05-04 03:00']
drw10     = mets[3]['2022-05-04 13:00':'2022-05-05 03:00']
drw_set   = mets[3]["2022-04-01 07:00":"2022-05-09 18:02"]

drwx = [drw0, drw1, drw2, drw3, drw4, drw5, drw6, drw7, drw8, drw9, drw10]
drwz = [mets[3].index.name, "Deerwood", drwx]

In [ ]:
## Pick specific dates and times you would like to look at for each site

print(mets[11].index.name)

ncm_whole = mets[11]
ncm_set   = mets[11]["2022-04-01 07:00":"2022-05-09 18:02"]

ncm0      = mets[11]['2022-04-01 13:00':'2022-04-02 03:00']
ncm1      = mets[11]['2022-04-05 13:00':'2022-04-06 03:00']
ncm2      = mets[11]['2022-04-06 13:00':'2022-04-07 03:00']
ncm3      = mets[11]['2022-04-09 13:00':'2022-04-10 03:00']
ncm4      = mets[11]['2022-04-17 13:00':'2022-04-18 03:00']
ncm5      = mets[11]['2022-04-23 13:00':'2022-04-24 03:00']
ncm6      = mets[11]['2022-04-24 13:00':'2022-04-25 03:00']
ncm7      = mets[11]['2022-04-27 13:00':'2022-04-28 03:00']
ncm8      = mets[11]['2022-05-01 13:00':'2022-05-02 03:00']
ncm9      = mets[11]['2022-05-03 13:00':'2022-05-04 03:00']
ncm10     = mets[11]['2022-05-04 13:00':'2022-05-05 03:00']

ncmx = [ncm0, ncm1, ncm2, ncm3, ncm4, ncm5, ncm6, ncm7, ncm8, ncm9, ncm10]
ncmz = [mets[11].index.name, "North Cow Mountain", ncmx]

In [ ]:
## Pick specific dates and times you would like to look at for each site

print(mets[9].index.name)

lbh_whole = mets[9]
lbh_set   = mets[9]['2021-05-01 13:00':'2021-06-01 03:00']

lbh0      = mets[9]['2021-05-01 13:00':'2021-05-02 03:00']
lbh1      = mets[9]['2021-05-03 13:00':'2021-05-04 03:00']
lbh2      = mets[9]['2021-05-04 13:00':'2021-05-05 03:00']
lbh3      = mets[9]['2021-05-05 13:00':'2021-05-06 03:00']
lbh4      = mets[9]['2021-05-07 13:00':'2021-05-08 03:00']
lbh5      = mets[9]['2021-05-08 13:00':'2021-05-09 03:00']
lbh6      = mets[9]['2021-05-10 13:00':'2021-05-11 03:00']
lbh7      = mets[9]['2021-05-11 13:00':'2021-05-12 03:00']
lbh8      = mets[9]['2021-05-12 13:00':'2021-05-13 03:00']
lbh9      = mets[9]['2021-05-13 13:00':'2021-05-14 03:00']
lbh10     = mets[9]['2021-05-30 13:00':'2021-05-31 03:00']

lbhx = [lbh0, lbh1, lbh2, lbh3, lbh4, lbh5, lbh6, lbh7, lbh8, lbh9, lbh10]
lbhz = [mets[9].index.name, "Lower Bathhouse", lbhx]

In [ ]:
lbhA = mets[9]["2021-04-21 07:00":"2021-05-01 07:00"][sr]
lbhB = mets[9]["2021-05-01 07:00":"2021-05-11 07:00"][sr] 
lbhC = mets[9]["2021-05-11 07:00":"2021-05-21 07:00"][sr] 
lbhD = mets[9]["2021-05-21 07:00":"2021-05-31 07:00"][sr] 

lbhG = [lbhA, lbhB, lbhC, lbhD]

In [ ]:
## Pick specific dates and times you would like to look at for each site

print(mets[5].index.name)

frc_whole = mets[5]
frc_set   = mets[5]["2022-05-01 07:00":"2022-06-04 18:02"]

frc0      = mets[5]['2022-05-01 13:00':'2022-05-02 03:00']
frc1      = mets[5]['2022-05-03 13:00':'2022-05-04 03:00']
frc2      = mets[5]['2022-05-04 13:00':'2022-05-05 03:00']
frc3      = mets[5]['2022-05-17 13:00':'2022-05-18 03:00']
frc4      = mets[5]['2022-05-19 13:00':'2022-05-20 03:00']
frc5      = mets[5]['2022-05-20 13:00':'2022-05-21 03:00']
frc6      = mets[5]['2022-05-22 13:00':'2022-05-23 03:00']
frc7      = mets[5]['2022-05-23 13:00':'2022-05-24 03:00']
frc8      = mets[5]['2022-05-24 13:00':'2022-05-25 03:00']
frc9      = mets[5]['2022-05-25 13:00':'2022-05-26 03:00']
frc10     = mets[5]['2022-05-31 13:00':'2022-06-01 03:00']

frcx = [frc0, frc1, frc2, frc3, frc4, frc5, frc6, frc7, frc8, frc9, frc10]
frcz = [mets[5].index.name, "Feather River College", frcx]

In [ ]:
frcA = mets[5]["2022-05-01 07:00":"2022-05-10 07:00"][sr]
frcB = mets[5]["2022-05-10 07:00":"2022-05-20 07:00"][sr]
frcC = mets[5]["2022-05-20 07:00":"2022-05-30 07:00"][sr]
frcD = mets[5]["2022-05-30 07:00":"2022-06-09 07:00"][sr]
frcG = [frcA, frcB, frcC, frcD]

In [ ]:
## Pick specific dates and times you would like to look at for each site

print(mets[0].index.name)

bcc_whole = mets[0]
bcc_set   = mets[0]["2022-04-01 07:00":"2022-05-09 18:02"]

bcc0      = mets[0]['2022-04-01 13:00':'2022-04-02 03:00']
bcc1      = mets[0]['2022-04-05 13:00':'2022-04-06 03:00']
bcc2      = mets[0]['2022-04-06 13:00':'2022-04-07 03:00']
bcc3      = mets[0]['2022-04-17 13:00':'2022-04-18 03:00']
bcc4      = mets[0]['2022-04-24 13:00':'2022-04-25 03:00']
bcc5      = mets[0]['2022-04-26 13:00':'2022-04-27 03:00']
bcc6      = mets[0]['2022-04-27 13:00':'2022-04-28 03:00']
bcc7      = mets[0]['2022-04-28 13:00':'2022-04-29 03:00']
bcc8      = mets[0]['2022-05-01 13:00':'2022-05-02 03:00']
bcc9      = mets[0]['2022-05-03 13:00':'2022-05-04 03:00']
bcc10     = mets[0]['2022-05-04 13:00':'2022-05-05 03:00']

bcc_whole = mets[0]["2022-04-01 07:00":"2022-07-07 00:00"]

bccx = [bcc0, bcc1, bcc2, bcc3, bcc4, bcc5, bcc6, bcc7, bcc8, bcc9, bcc10]
bccz = [mets[0].index.name, "Boyes Creek Canyon", bccx]

In [ ]:
## Pick specific dates and times you would like to look at for each site

print(mets[10].index.name)

nbb_whole = mets[10]
nbb_set   = mets[10]["2022-04-01 07:00":"2022-05-09 18:02"]

nbb0      = mets[10]['2022-04-01 13:00':'2022-04-02 03:00']
nbb1      = mets[10]['2022-04-05 13:00':'2022-04-06 03:00']
nbb2      = mets[10]['2022-04-06 13:00':'2022-04-07 03:00']
nbb3      = mets[10]['2022-04-09 13:00':'2022-04-10 03:00']
nbb4      = mets[10]['2022-04-17 13:00':'2022-04-18 03:00']
nbb5      = mets[10]['2022-04-23 13:00':'2022-04-24 03:00']
nbb6      = mets[10]['2022-04-24 13:00':'2022-04-25 03:00']
nbb7      = mets[10]['2022-04-26 13:00':'2022-04-27 03:00']
nbb8      = mets[10]['2022-05-01 13:00':'2022-05-02 03:00']
nbb9      = mets[10]['2022-05-03 13:00':'2022-05-04 03:00']
nbb10     = mets[10]['2022-05-04 13:00':'2022-05-05 03:00']

nbbx = [nbb0, nbb1, nbb2, nbb3, nbb4, nbb5, nbb6, nbb7, nbb8, nbb9, nbb10]
nbbz = [mets[10].index.name, "New Bullards Bar Dam", nbbx]

In [ ]:
## Pick specific dates and times you would like to look at for each site

print(mets[-3].index.name)

sod_whole = mets[-3]
sod_set   = mets[-3]["2022-04-01 07:00":"2022-05-09 18:02"]

sod0      = mets[-3]['2022-04-05 13:00':'2022-04-06 03:00']
sod1      = mets[-3]['2022-04-06 13:00':'2022-04-07 03:00']
sod2      = mets[-3]['2022-04-07 13:00':'2022-04-08 03:00']
sod3      = mets[-3]['2022-04-12 13:00':'2022-04-13 03:00']
sod4      = mets[-3]['2022-04-13 13:00':'2022-04-14 03:00']
sod5      = mets[-3]['2022-04-14 13:00':'2022-04-15 03:00']
sod6      = mets[-3]['2022-04-15 13:00':'2022-04-16 03:00']
sod7      = mets[-3]['2022-04-29 13:00':'2022-04-30 03:00']
sod8      = mets[-3]['2022-05-01 13:00':'2022-05-02 03:00']
sod9      = mets[-3]['2022-05-02 13:00':'2022-05-03 03:00']
sod10     = mets[-3]['2022-05-04 13:00':'2022-05-05 03:00']

sodx = [sod0, sod1, sod2, sod3, sod4, sod5, sod6, sod7, sod8, sod9, sod10]
sodz = [mets[-3].index.name, "Seven Oaks Dam", sodx]

In [ ]:
## Pick specific dates and times you would like to look at for each site

print(mets[7].index.name)

hdc_whole = mets[7]
hdc_set   = mets[7]["2022-06-03 07:00":"2022-07-07 00:00"]

hdc0      = mets[7]['2022-06-06 13:00':'2022-06-07 03:00']
hdc1      = mets[7]['2022-06-13 13:00':'2022-06-14 03:00']
hdc2      = mets[7]['2022-06-14 13:00':'2022-06-15 03:00']
hdc3      = mets[7]['2022-06-20 13:00':'2022-06-21 03:00']
hdc4      = mets[7]['2022-06-21 13:00':'2022-06-22 03:00']
hdc5      = mets[7]['2022-06-25 13:00':'2022-06-26 03:00']
hdc6      = mets[7]['2022-06-26 13:00':'2022-06-27 03:00']
hdc7      = mets[7]['2022-06-28 13:00':'2022-06-29 03:00']
hdc8      = mets[7]['2022-06-29 13:00':'2022-06-30 03:00']
hdc9      = mets[7]['2022-06-30 13:00':'2022-07-01 03:00']
hdc10     = mets[7]['2022-07-01 13:00':'2022-07-02 03:00']

hdcx = [hdc0, hdc1, hdc2, hdc3, hdc4, hdc5, hdc6, hdc7, hdc8, hdc9, hdc10]
hdcz = [mets[7].index.name, "Hell's Delight Canyon", hdcx]

In [ ]:
hdcA = mets[7]["2022-06-03 07:00":"2022-06-10 07:00"][sr]
hdcB = mets[7]["2022-06-10 07:00":"2022-06-20 07:00"][sr]
hdcC = mets[7]["2022-06-20 07:00":"2022-06-30 07:00"][sr]
hdcD = mets[7]["2022-06-30 07:00":"2022-07-06 07:00"][sr]
hdcG = [hdcA, hdcB, hdcC, hdcD]

In [ ]:
## Pick specific dates and times you would like to look at for each site

print(mets[-5].index.name)

pvn_whole = mets[-5]
pvn_set   = mets[-5]["2022-06-03 07:00":"2022-07-07 00:00"]

pvn0      = mets[-5]['2022-04-01 13:00':'2022-04-02 03:00']
pvn1      = mets[-5]['2022-04-05 13:00':'2022-04-06 03:00']
pvn2      = mets[-5]['2022-04-06 13:00':'2022-04-07 03:00']
pvn3      = mets[-5]['2022-04-09 13:00':'2022-04-10 03:00']
pvn4      = mets[-5]['2022-04-17 13:00':'2022-04-18 03:00']
pvn5      = mets[-5]['2022-04-24 13:00':'2022-04-25 03:00']
pvn6      = mets[-5]['2022-04-26 13:00':'2022-04-27 03:00']
pvn7      = mets[-5]['2022-04-27 13:00':'2022-04-28 03:00']
pvn8      = mets[-5]['2022-05-01 13:00':'2022-05-02 03:00']
pvn9      = mets[-5]['2022-05-03 13:00':'2022-05-04 03:00']
pvn10     = mets[-5]['2022-05-04 13:00':'2022-05-05 03:00']

pvnx = [pvn0, pvn1, pvn2, pvn3, pvn4, pvn5, pvn6, pvn7, pvn8, pvn9, pvn10]
pvnz = [mets[-5].index.name, "Potter Valley North", pvnx]

In [ ]:
## Creating a list of all sites and their dates/times.
allsites = (gpox, porx, bvsx, dlax, skyx, wpox, wdgx, hrdx, catx, drwx, ncmx, lbhx, frcx, \
            bccx, nbbx, sodx, hdcx, pvnx)
allsets = (gpo_set, por_set, bvs_set, dla_set, sky_set, wpo_set, wdg_set, hrd_set, cat_set, \
           drw_set, ncm_set, lbh_set, frc_set, bcc_set, nbb_set, sod_set, hdc_set, pvn_set)
allsitez = (gpoz, porz, bvsz, dlaz, skyz, wpoz, wdgz, hrdz, catz, drwz, ncmz, lbhz, frcz, \
            bccz, nbbz, sodz, hdcz, pvnz)

In [ ]:
# Making sure they're all the same (i.e., that no sites have been left out)
print(len(allsites), len(allsets), len(allsitez))

In [ ]:
## EXAMPLE PLOT:
SLR(wdg_whole, ' from '+str(wdg_whole.index.date[0])+' to '+str(wdg_whole.index.date[-1]))

In [ ]:
#most recent site to look at
for i in range(len(allsets)):
    SLR(allsets[i], 'at '+str(allsitez[i][1])+' from '+str(allsets[i].index.date[0])+' to '+str(allsets[i].index.date[-1]))
    

In [ ]:
#To mass print all subsets/days of solar rad into separate graphs:

print(allsitez[7][1])
allSLR((allsitez[7])[2])

In [ ]:
## Creating lists for specific dates, times, and quantities where solar rad is at its peak.
def dvar(locx):
    h = []
    j = []
    k = []
    for i in range(len(locx)):
        F = (locx[i])[sr].idxmax()
        h = np.append(h, F.date())
    for i in range(len(locx)):
        F = (locx[i])[sr].idxmax()
        j = np.append(j, F.time())
    for i in range(len(locx)):
        F = (locx[i])[sr].idxmax()
        k = np.append(k, locx[i][sr].max())
    return(h,j,k)

## For an easy copy/paste of said list
def printy(site):
    h, j, k = dvar(site)
    for i in range(len(site)):
        print(h[i])
    print('\n')
    for i in range(len(site)):
        print(j[i])
    print('\n')
    for i in range(len(site)):
        print(k[i])

In [ ]:
printy(allsites[-1])
hi = csky(allsitez[0][2])

In [ ]:
# Calculating percentage error of maximum solar rad. to top 5 values in each day pe = |(m-a)/a| * 100

def percerr(name, day, var):
    maxi = np.max(day[var])
    res = day[var].nlargest(n=5)
    #print(res)
    avg = pd.DataFrame.mean(res)
    per = abs(((maxi-avg)/avg))*100
    print(name)
    print("Max: ", maxi)
    #format to three decimal places
    form_str = "{:.3f}".format(per)
    float_val = float(form_str)
    print(str(float_val)+'%')
    return per

def qerr(name, site):
    P = []
    V = np.array([], dtype = object)
    J = []
    K = []
    L = []
    M = []
    x = 0
    print('Using max. value of timeframe:')
    for i in range(len(site)):
        j = (i+1)
        J = np.append(J, j)
        v = name+' Day '+str(i+1)
        V = np.append(V, v)
        per = percerr(v, site[i], sr)
        if per <= 0.33:
            x += 1
            K = np.append(K, per)
            L = np.append(L, x)
        elif per > 0.33:
            M = np.append(M, per)
        P = np.append(P, per)    
    pT = np.mean(np.float16(P))
    print('Overall percentage error @ '+name+' is:', str(pT)+'%\n')
    return(name, V, P, pT, J, K, L, M)

def mpercerr(name, day, var, mod):
    res = day[var].nlargest(n=5)
    #print(res)
    avg = pd.DataFrame.mean(res)
    per = abs(((mod-avg)/avg))*100
    print(name)
    print("Model: ", mod)
    #format to three decimal places
    form_str = "{:.3f}".format(per)
    float_val = float(form_str)
    print(str(float_val)+'%')
    return per
        
def mqerr(name, site, model):
    Q = []
    V = np.array([], dtype = object)
    J = []
    print('Using the model from ClearSky:')
    for i in range(len(site)):
        j = (i+1)
        J = np.append(J, j)
        v = name+' Day '+str(i+1)
        V = np.append(V, v)
        per = mpercerr(v, site[i], sr, model[i])
        Q = np.append(Q, per)
    qT = np.mean(np.float16(Q))
    print('Overall percentage error @ '+name+' in relation to ClearSky model is:', str(qT)+'%\n')
    return(name, V, Q, qT, J)

def Perplt(site, x):
    if(site[2]).max() > 0.33: 
        plt.bar(site[6], site[5], color = ('blue','cornflowerblue','lightgreen','gold','orange','tomato'))
        plt.xticks(range(len(site[6])+1))
    else:
        plt.bar(site[4], site[2], color = ('blue','cornflowerblue','lightgreen','gold','orange','tomato'))
        plt.xticks(range(len(site[2])+1))
    plt.title(str(site[0])+' Solar Rad Percentage Errors', fontweight = 'bold', fontsize = '18')
    plt.ylabel('Percentage Error (%)', fontweight = 'bold', fontsize = '17')
    plt.yticks(levels)
    plt.show()
    print('Where PE = |(M-A)/A|*100, the errors are:', np.float64(site[2]))
    if(np.array(site[7]).size > 0):
        print('At '+str(site[0])+', these percentage values were anomalous to the other days:'+str(site[7]))
    else:
        return
    
def MPerplt(site, x):
    plt.bar(site[4], site[2], color = ('blue','cornflowerblue','lightgreen','gold','orange','tomato'))
    plt.title(str(site[0])+' Solar Rad Percentage Errors w/ Model', fontweight = 'bold', fontsize = '18')
    plt.xlabel(str(x), fontweight = 'bold', fontsize = '17')
    plt.xticks(range(len(site[2])+1))
    plt.ylabel('Percentage Error (%)', fontweight = 'bold', fontsize = '17')
    plt.show()
    print('Where PE = |(M-A)/A|*100, the errors are:', np.float64(site[2]))
    
def twodec(value):
    form_str = "{:.2f}".format(value)
    fl_val = float(form_str)

    return(fl_val)

In [ ]:
levels = np.arange(0, 0.35, step = 0.05)
mgpo = [881, 893, 900, 926, 940, 962, 984, 970, 980, 977, 982]

gran = qerr('GPO', gpox)
granm = mqerr('GPO', gpox, mgpo)

Perplt(gran, 'Days')
MPerplt(granm, 'Days')

In [ ]:
levels = np.arange(0, 0.35, step = 0.05)
mpor = [886, 901, 910, 925, 949, 947, 967, 963, 983, 986, 982]

port = qerr('POR', porx)
portm = mqerr('POR', porx, mpor)

Perplt(port, 'Days')
MPerplt(portm, 'Days')

In [ ]:
levels = np.arange(0, 0.35, step = 0.05)
mbvs = [844, 878, 880, 905, 907, 926, 924, 938, 949, 955, 944]

brow = qerr('BVS', bvsx)
browm = mqerr('BVS', bvsx, mbvs)

Perplt(brow, 'Days')
MPerplt(browm, 'Days')

In [ ]:
levels = np.arange(0, 0.35, step = 0.05)
mdla = [872, 883, 892, 903, 925, 951, 974, 976, 967]

down = qerr('DLA', dlax)
downm = mqerr('DLA', dlax, mdla)

Perplt(down, 'Days')
MPerplt(downm, 'Days')

In [ ]:
levels = np.arange(0, 0.35, step = 0.05)
msky = [863, 871, 852, 890, 917, 929, 944, 943, 969, 962, 956]

skyl = qerr('SKY', skyx)
skylm = mqerr('SKY', skyx, msky)
print(skyl[5])
print(skyl[6])
## NOTE: 3, 5, & 7 high error
print(str(twodec(skyl[7][0]))+'%', str(twodec(skyl[7][1]))+'%', str(twodec(skyl[7][2]))+'%')

Perplt(skyl, 'Days')
MPerplt(skylm, 'Days')

In [ ]:
mwest = [864, 880, 886, 906, 937, 918, 949, 966, 949, 979, 970]

west = qerr('WPO', wpox)
westm = mqerr('WPO', wpox, mwest)

Perplt(west, 'Days')
MPerplt(westm, 'Days')
 
#print(west[5])
#westb = (west[2])[np.arange(len(west[2]))!=3]  # b = array([9, 8, 7, 5, 4, 3, 2, 1, 0])
#print(westb)

In [ ]:
mwdy = [892, 883, 911, 916, 939, 942, 935, 945, 957, 976, 959]

wind = qerr('WDG', wdgx)
windm = mqerr('WDG', wdgx, mwdy)

Perplt(wind, 'Days')
MPerplt(windm, 'Days')

print("{:.3f}".format((wind[2])[3])+'%')

In [ ]:
mhen = [913, 927, 941, 945, 979, 972, 982, 1006, 994, 1000, 1001]

henn = qerr('HRD', hrdx)
hennm = mqerr('HRD', hrdx, mhen)

Perplt(henn, 'Days')
MPerplt(hennm, 'Days')

In [ ]:
mcat = [893, 937, 944, 957, 953, 955, 978, 970, 957, 962, 962]

cata = qerr('CAT', catx)
catam = mqerr('CAT', catx, mcat)

Perplt(cata, 'Days')
MPerplt(catam, 'Days')

In [ ]:
mdrw = 855, 872, 879, 894, 939, 933, 933, 937, 962, 962, 943

deer = qerr('DRW', drwx)
deerm = mqerr('DRW', drwx, mdrw)

Perplt(deer, 'Days')
MPerplt(deerm, 'Days')

In [ ]:
mncm = 864, 896, 893, 912, 921, 952, 949, 955, 963, 976, 963

cow = qerr('NCM', ncmx)
cowm = mqerr('NCM', ncmx, mncm)

Perplt(cow, 'Days')
MPerplt(cowm, 'Days')

In [ ]:
mlbh = 993, 995, 984, 977, 993, 1014, 1011, 1004, 1006, 998, 1017

bath = qerr('LBH', lbhx)
bathm = mqerr('LBH', lbhx, mlbh)

Perplt(bath, 'Days')
MPerplt(bathm, 'Days')

In [ ]:
mfrc = 969, 967, 969, 986, 978, 1018, 993, 987, 996, 973, 1008

feat = qerr('FRC', frcx)
featm = mqerr('FRC', frcx, mfrc)

Perplt(feat, 'Days')
MPerplt(featm, 'Days')

In [ ]:
mbcc = 849, 880, 860, 930, 941, 937, 942, 949, 936, 945, 940

boy = qerr('BCC', bccx)
boym = mqerr('BCC', bccx, mbcc)

Perplt(boy, 'Days')
MPerplt(boym, 'Days')

In [ ]:
mnbb = 858, 866, 886, 900, 913, 932, 946, 946, 977, 962, 957

bar = qerr('NBB', nbbx)
barm = mqerr('NBB', nbbx, mnbb)

Perplt(bar, 'Days')
MPerplt(barm, 'Days')

In [ ]:
msod = 914, 929, 961, 989, 988, 982, 970, 990, 992, 980, 1006

sev = qerr('SOD', sodx)
sevm = mqerr('SOD', sodx, msod)

Perplt(sev, 'Days')
MPerplt(sevm, 'Days')

In [ ]:
mhdc = 979, 999, 994, 992, 1000, 983, 1005, 983, 987, 975, 974

hell = qerr('HDC', hdcx)
hellm = mqerr('HDC', hdcx, mhdc)

Perplt(hell, 'Days')
MPerplt(hellm, 'Days')

In [ ]:
mpvn = 843, 875, 866, 901, 920, 931, 924, 932, 935, 943, 949

pot = qerr('PVN', pvnx)
potm = mqerr('PVN', pvnx, mpvn)

Perplt(pot, 'Days')
MPerplt(potm, 'Days')

In [ ]:
allmeans = [gran, port, brow, down, skyl, west, wind, henn, cata, deer, cow, bath, feat, boy, bar, sev, hell, pot]
mallmeans= [granm, portm, browm, downm, skylm, westm, windm, hennm, catam, deerm, cowm, bathm, featm, \
            boym, barm, sevm, hellm, potm]

In [ ]:
def allsite(site):
    g = []
    h = []
    j = []
    k = []
    for i in range(len(site)):
        if(site[i][3] < 50):
            g = np.append(g, site[i][0])
            h = np.append(h, site[i][3])
        else:
            j = np.append(j, site[i][0])
            k = np.append(k, site[i][3])
    return(g, h, j, k)   

def plotALL(site, z):
    x, y, a, b = allsite(site)
    plt.bar(x, y, color = ('blue','cornflowerblue','lightgreen','gold','orange','tomato'))
    plt.title('All Sites Solar Rad Percentage Errors'+str(z), fontweight = 'bold', fontsize = '18')
    plt.xlabel('Site Locations', fontweight = 'bold', fontsize = '17')
    plt.xticks(x)
    plt.ylabel('Percentage Error (%)', fontweight = 'bold', fontsize = '17')
    plt.show()
    print('Where PE = |(M-A)/A|*100, the errors are:', np.float64(y))
    if(np.array(a).size > 0):
        print('At '+str(a)+', these percentage values were anomalous to the other days:'+str(b))
    else:
        return
#    
def anom(sites, z):
    x, y, a, b = allsite(sites)
    plt.bar(a, b, color = ('blue','cornflowerblue','lightgreen','gold','orange','tomato'))
    plt.title('All Sites Solar Rad Percentage Errors'+str(z), fontweight = 'bold', fontsize = '18')
    plt.xlabel('Site Locations', fontweight = 'bold', fontsize = '17')
    plt.xticks(x)
    plt.ylabel('Percentage Error (%)', fontweight = 'bold', fontsize = '17')
    plt.show()
    print('Where PE = |(M-A)/A|*100, the errors are:', np.float64(b))
    if(np.array(a).size > 0):
        print('At '+str(a)+', these percentage values were anomalous to the other days:'+str(b))
    else:
        return

In [ ]:
plotALL(allmeans, ' w/o model')
plotALL(mallmeans, ' w/ Clearsky model')

In [ ]:
anomaly = [windm, cowm, featm, sevm]
anom(anomaly, ' w/ Anomalous Solar Rad')

In [ ]:
for i in range(len(hrdx)):
    print(str(hrdx[i].index.date[0]))

In [ ]:
z = len(hrd0)
w = len(impvals)
v = len(henny)
print(z,w,v)

for a in range(z):
    if hrd0[sr][a] == hrd0[sr].max():
        for c in range(w):
            print(hrd0[sr].idxmax())
            print(hrd0[impvals[c]][b])
    else:
            continue
            
print(hrd0[sr].max())

In [ ]:
hrd4y      = mets[8]['2022-04-12 13:00':'2022-04-13 03:00']
hrd8y      = mets[8]['2022-05-01 13:00':'2022-05-02 03:00']

hrdy = [hrd0, hrd1, hrd2, hrd3, hrd4y, hrd5, hrd6, hrd7, hrd8y, hrd9, hrd10]

allsitesy = ()
allsets = ()
allsitesy = (gpox, porx, bvsx, dlax, skyx, wpox, wdgx, hrdy, catx, drwx, ncmx, lbhx, frcx, \
            bccx, nbbx, sodx, hdcx, pvnx)

In [ ]:
import matplotlib.dates as mdates

def allday(site, loc):
    times = pd.date_range('2015-05-01 06:00', periods = 421, freq = '2T')

    plt.rcParams["figure.figsize"] = [20.00, 8.0]
    plt.rcParams["figure.autolayout"] = True
    fig, ax = plt.subplots(1)
    fig.autofmt_xdate()
    for i in range(len(site)): 
        plt.plot(times, site[i][sr])
    plt.xlabel('Time of Day (PDT)', fontweight = 'bold', fontsize = 16)
    plt.ylabel('Solar Radiation (W/m\N{superscript two})', fontweight = 'bold', fontsize = 16)
    xfmt = mdates.DateFormatter('%H:%M')
    ax.xaxis.set_major_formatter(xfmt)
    plt.title(str(loc)+' Solar Rad', fontweight = 'bold', fontsize = '20')
    plt.show()

In [ ]:
for i in range(len(allsites)):
    allday(allsitesy[i], allsitez[i][1])